# select conda envirnment coz of FAISS library

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('datasetUpdate.csv')

In [3]:
df

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,27205,Inception,"Action,Science Fiction,Adventure",en,"Cobb, a skilled thief who commits corporate es...",111.757,7/15/2010,8.4,31917
1,157336,Interstellar,"Adventure,Drama,Science Fiction",en,The adventures of a group of explorers who mak...,191.220,11/5/2014,8.4,28920
2,155,The Dark Knight,"Drama,Action,Crime,Thriller",en,Batman raises the stakes in his war on crime. ...,82.230,7/14/2008,8.5,27925
3,24428,The Avengers,"Science Fiction,Action,Adventure",en,When an unexpected enemy emerges and threatens...,278.980,4/25/2012,7.7,27250
4,293660,Deadpool,"Action,Adventure,Comedy",en,The origin story of former Special Forces oper...,230.309,2/9/2016,7.6,27028
...,...,...,...,...,...,...,...,...,...
9995,28569,Splendor in the Grass,"Drama,Romance",en,A fragile Kansas girl's unrequited and forbidd...,8.734,10/10/1961,7.6,200
9996,4952,Pillow Talk,"Comedy,Romance",en,A man and woman share a telephone line and des...,6.256,10/7/1959,7.2,200
9997,15515,The 7th Voyage of Sinbad,"Action,Adventure,Family,Fantasy",en,"When a princess is shrunken by an evil wizard,...",10.421,12/5/1958,6.9,200
9998,9077,Tarantula,"Horror,Science Fiction,Thriller",en,A lone-wolf scientist sequestered in a mansion...,10.909,12/14/1955,6.4,200


In [4]:
df.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [5]:
df.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [6]:
df=df[['title', 'overview', 'genre','release_date']]

In [7]:
df

,title,overview,genre,release_date
0,Inception,"Cobb, a skilled thief who commits corporate es...","Action,Science Fiction,Adventure",7/15/2010
1,Interstellar,The adventures of a group of explorers who mak...,"Adventure,Drama,Science Fiction",11/5/2014
2,The Dark Knight,Batman raises the stakes in his war on crime. ...,"Drama,Action,Crime,Thriller",7/14/2008
3,The Avengers,When an unexpected enemy emerges and threatens...,"Science Fiction,Action,Adventure",4/25/2012
4,Deadpool,The origin story of former Special Forces oper...,"Action,Adventure,Comedy",2/9/2016
...,...,...,...,...
9995,Splendor in the Grass,A fragile Kansas girl's unrequited and forbidd...,"Drama,Romance",10/10/1961
9996,Pillow Talk,A man and woman share a telephone line and des...,"Comedy,Romance",10/7/1959
9997,The 7th Voyage of Sinbad,"When a princess is shrunken by an evil wizard,...","Action,Adventure,Family,Fantasy",12/5/1958
9998,Tarantula,A lone-wolf scientist sequestered in a mansion...,"Horror,Science Fiction,Thriller",12/14/1955


In [8]:
def create_textual_representation(row):
    textual_representation=f"""
Title: {row['title']},
Year: {row['release_date']},
Genre: {row['genre']},

Description: {row['overview']}"""
    return textual_representation

In [9]:
df_copy = df.copy()
df_copy['textual_representation'] = df_copy.apply(create_textual_representation, axis=1)

In [10]:
print(df_copy['textual_representation'].values[0])


Title: Inception,
Year: 7/15/2010,
Genre: Action,Science Fiction,Adventure,

Description: Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person's idea into a target's subconscious.


In [11]:
import faiss
print(faiss.__version__)
import requests
import numpy as np

1.7.4


In [12]:
dim = 4096

index = faiss.IndexFlatL2(dim)

x = np.zeros((len(df_copy['textual_representation']), dim), dtype = 'float32')

In [13]:
#x

In [14]:
for i, representation in enumerate(df_copy['textual_representation']):
    if i % 1000 == 0:
        print("Processing textual representation", i)
        
        res = requests.post('http://localhost:11434/api/embeddings', 
                            json={'model': 'llama2',
                            'prompt': representation,
                                }
                            )
        embedding = res.json()['embedding']
        
        x[i] = np.array(embedding)
        
index.add(x)

Processing textual representation 0
Processing textual representation 1000
Processing textual representation 2000
Processing textual representation 3000
Processing textual representation 4000
Processing textual representation 5000
Processing textual representation 6000
Processing textual representation 7000
Processing textual representation 8000
Processing textual representation 9000


In [15]:
faiss.write_index(index, 'index')

In [16]:
index = faiss.read_index('index')

In [36]:
df[df.title.str.contains('shutter')]

,title,overview,genre,release_date


In [18]:
favorite_movie = df.iloc[0]

In [19]:
favorite_movie

title                                                   Inception
overview        Cobb, a skilled thief who commits corporate es...
genre                            Action,Science Fiction,Adventure
release_date                                            7/15/2010
Name: 0, dtype: object

In [42]:
recommend_movie = """
Title: The Plot,
Year: 21 June 2024,
Genre: 

Description: The Plot is a 2024 South Korean crime thriller film directed by Lee Yo-sup, starring Gang Dong-won. An adaptation of the Hong Kong film, Accident, it tells the story of Yeong-il, a designer who orchestrates a commissioned murder as a perfect accidental death, and gets caught up in an unexpected incident
"""

In [43]:
res = requests.post('http://localhost:11434/api/embeddings', 
                            json={'model': 'llama2',
                            'prompt': recommend_movie,
                                }
                            )

In [44]:
embedding = np.array([res.json()['embedding']], dtype = 'float32')

D ,I = index.search(embedding, 5)

In [45]:
best_matches = np.array(df_copy['textual_representation'])[I.flatten()]

In [46]:
for match in best_matches:
    print('Next Movie')
    print(match)
    print()

Next Movie

Title: Interstellar,
Year: 11/5/2014,
Genre: Adventure,Drama,Science Fiction,

Description: The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.

Next Movie

Title: The Dark Knight,
Year: 7/14/2008,
Genre: Drama,Action,Crime,Thriller,

Description: Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker.

Next Movie

Title: The Avengers,
Year: 4/25/2012,
Genre: Science Fiction,Action,Adventure,

Description: When an unexpected enemy emerges and threatens global safety and security, Nick Fury, dire

In [28]:
import pickle

In [29]:
pickle.dump(best_matches, open('best_matches.pkl', 'wb'))

In [30]:
pickle.load(open('best_matches.pkl', 'rb'))

array(['\nTitle: Inception,\nYear: 7/15/2010,\nGenre: Action,Science Fiction,Adventure,\n\nDescription: Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.',
       '\nTitle: Interstellar,\nYear: 11/5/2014,\nGenre: Adventure,Drama,Science Fiction,\n\nDescription: The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.',
       '\nTitle: The Dark Knight,\nYear: 7/14/2008,\nGenre: Drama,Action,Crime,Thriller,\n\nDescription: Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the stre